In [1]:
import os
import numpy as np
import netCDF4 as nc
import geopandas as gpd
from tqdm import tqdm
import rasterio
from osgeo import gdal, osr
from rasterstats import zonal_stats

# 1. Convert NC to raster

In [2]:
def nc_to_geotiff(read_path, save_path):
    nc_file = nc.Dataset(read_path)
    lat = nc_file.variables['latitude'][:]
    lon = nc_file.variables['longitude'][:]
    PM25 = np.asanyarray(nc_file.variables['PM25'])

    # get the spatial range of the netcdf
    Lonmin, Latmax, Lonmax, Latmin = [lon.min(), lat.max(), lon.max(), lat.min()]

    # calculate the resolution
    Num_lat = len(lat)
    Num_lon = len(lon)
    Lat_res = (Latmax - Latmin) / (float(Num_lat) - 1)
    Lon_res = (Lonmax - Lonmin) / (float(Num_lon) - 1)

    # create the tif file and save it into the virtual file system in memory
    driver = gdal.GetDriverByName('GTiff')
    out_tif = driver.Create('/vsimem/PM25.tif', Num_lon, Num_lat, 1, gdal.GDT_Float32)

    # set the spatial range of the tif file
    geotransform = (Lonmin, Lon_res, 0.0, Latmax, 0.0, -Lat_res)
    out_tif.SetGeoTransform(geotransform)

    # set the projection system
    prj = osr.SpatialReference()
    prj.ImportFromEPSG(4326)
    out_tif.SetProjection(prj.ExportToWkt())

    # check is the data are flipped and correct the data if yes
    if lat[0] <= lat[-1]: 
        PM25 = PM25[::-1]
    else:
        pass

    # write data into tif and close the file
    out_tif.GetRasterBand(1).WriteArray(PM25)
    # transform the projection to 3035 and save
    gdal.Warp(save_path, out_tif, srcSRS='EPSG:4326', dstSRS='EPSG:3035')
    out_tif.FlushCache() 
    out_tif = None

In [ ]:
read_folder = '/data/xiang/1-Data/PM2.5/netcdf'
save_folder = '/data/xiang/1-Data/PM2.5/raster'

if __name__ == '__main__':
    for netc in tqdm(os.listdir(read_folder)):
        if netc.endswith('.nc'):
            read_path = read_folder + r'/' + netc
            save_path = save_folder + r'/' + netc[:-2] + '.tif'
            nc_to_geotiff(read_path, save_path)

  0%|                                                    | 0/25 [00:00<?, ?it/s]

# 2. Zonal statistics

In [1]:
# read nuts shp and PM2.5 raster data
nuts = gpd.read_file('/data/xiang/1-Data/NUTS/eu.shp')
PM_ras = rasterio.open('/data/xiang/1-Data/PM2.5/raster/V6G02_2009.tif')

NameError: name 'gpd' is not defined

In [9]:
rasterio.warp.reproject?

Signature:
rasterio.warp.reproject(
    source,
    destination=None,
    src_transform=None,
    gcps=None,
    rpcs=None,
    src_crs=None,
    src_nodata=None,
    dst_transform=None,
    dst_crs=None,
    dst_nodata=None,
    dst_resolution=None,
    src_alpha=0,
    dst_alpha=0,
    resampling=<Resampling.nearest: 0>,
    num_threads=1,
    init_dest_nodata=True,
    warp_mem_limit=0,
    **kwargs,
)
Docstring:
Reproject a source raster to a destination raster.

If the source and destination are ndarrays, coordinate reference
system definitions and affine transformation parameters or ground
control points (gcps) are required for reprojection.

If the source and destination are rasterio Bands, shorthand for
bands of datasets on disk, the coordinate reference systems and
transforms or GCPs will be read from the appropriate datasets.

Parameters
------------
source: ndarray or Band
    The source is a 2 or 3-D ndarray, or a single or a multiple
    Rasterio Band object. The dimension

In [ ]:
zonal_stats(nuts, PM_ras, stats='mean')